## <font color=green> Aplicação de funções e mapeamento

In [1]:
import pandas as pd
import numpy as np

- Uma operação frequente consiste em aplicar uma função em arrays unidimensionais para cada coluna ou linha. O método apply faz exatamente isso.

In [2]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)), 
                        index=['a','c','d'], 
                        columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


- Nesse caso, a função f, que calcula a diferença entre o máximo e o mínimo de uma Series, é chamada uma vez em cada coluna de frame. O resultado é uma Series com as colunas de frame como seu índice.

In [3]:
f = lambda x: x.max() - x.min()
frame.apply(f)

Ohio          6
Texas         6
California    6
dtype: int64

- Se você passar axis='columns' para apply, a função será chamada uma vez por linha

In [4]:
frame.apply(f, axis='columns')

a    2
c    2
d    2
dtype: int64

- Podemos atribuir uma nova coluna que recebe o valor do processamento do apply

In [5]:
frame['total'] = frame.apply(lambda x: x.sum(), axis='columns')
frame

,Ohio,Texas,California,total
a,0,1,2,3
c,3,4,5,12
d,6,7,8,21


In [6]:
frame['category'] = frame['total'].apply(lambda x: 'impar' if x%2 else 'par')
frame

,Ohio,Texas,California,total,category
a,0,1,2,3,impar
c,3,4,5,12,par
d,6,7,8,21,impar


- A função passada para apply não precisa devolver um valor escalar. Ela também pode devolver uma Series com múltiplos valores

In [7]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f)

,Ohio,Texas,California,total,category
min,0,1,2,3,impar
max,6,7,8,21,par


- Suponha que você quisesse calcular uma string formatada para cada valor de ponto flutuante em frame. Isso poderia ser feito com applymap
- O motivo para o nome applymap está no fato de Series ter um método map para aplicar uma função em todos os elementos

In [8]:
numbers = pd.DataFrame(np.random.randn(4,3), columns=list('abc'), index=['c1', 'c2', 'c3', 'c4'])
numbers

,a,b,c
c1,-1.106564,-1.384582,0.324463
c2,0.398629,-1.767336,1.515648
c3,-1.816829,0.212401,-0.096618
c4,0.517339,-0.032937,-0.270520


In [9]:
format = lambda x: '%.2f' % x
numbers.applymap(format)

,a,b,c
c1,-1.11,-1.38,0.32
c2,0.40,-1.77,1.52
c3,-1.82,0.21,-0.10
c4,0.52,-0.03,-0.27


In [10]:
numbers['c'].map(format)

c1     0.32
c2     1.52
c3    -0.10
c4    -0.27
Name: c, dtype: object

- Suponha que você quisesse exibir os valores menores que 0 na cor vermelha e o restante na cor azul. Isso também pode ser feito com applymap

In [11]:
def color_df(val):
    color = 'red' if val < 0 else 'blue'
    return 'color: ' + color

In [12]:
numbers.style.applymap(color_df)

,a,b,c
c1,-1.106564,-1.384582,0.324463
c2,0.398629,-1.767336,1.515648
c3,-1.816829,0.212401,-0.096618
c4,0.517339,-0.032937,-0.270520


## <font color=green> Ordenação e classificação

- Para ordenar de modo lexicográfico pelo índice da linha ou da coluna, utilize o método sort_index, que devolve um novo objeto ordenado

In [13]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [14]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

- Com um DataFrame, você pode ordenar pelo índice em qualquer eixo

In [15]:
frame = pd.DataFrame(np.arange(8).reshape((2,4)), index=['three', 'one'], columns=['d', 'a', 'b', 'c'])

In [16]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [17]:
frame.sort_index(axis='columns')

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


- Os dados são ordenados em ordem crescente por padrão, mas podem ser ordenados também em ordem decrescente

In [18]:
frame.sort_index(axis='columns', ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


- Para ordenar uma Series de acordo com seus valores, utiliza o seu método sort_values

In [19]:
obj = pd.Series([4,7,-3,2])
obj

0    4
1    7
2   -3
3    2
dtype: int64

In [20]:
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

- Qualquer valor indicativo de ausência será ordenado no final da Series, por padrão

In [21]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj

0    4.0
1    NaN
2    7.0
3    NaN
4   -3.0
5    2.0
dtype: float64

In [22]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

- Quando ordenar um DataFrame, você poderá usar os dados de uma ou mais colunas como chaves de ordenação. Para isso, passe um ou mais nomes de coluna para a opção by de sort_values

In [23]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [24]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


- Para ordenar de acordo com várias colunas, passe uma lista de nomes

In [25]:
frame.sort_values(by=['a','b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


- Outra forma de ordenação é através dos métodos nlargest e nsmallest que retornam respectivamente os maiors e menores valores de uma ou mais coluna.

In [26]:
df_movie = pd.read_csv("datasets/Movie+Assignment+Data.csv", usecols=["Title","title_year","IMDb_rating","budget","Gross"])
df_movie

,Title,title_year,budget,Gross,IMDb_rating
0,La La Land,2016,30000000,151101803,8.2
1,Zootopia,2016,150000000,341268248,8.1
2,Lion,2016,12000000,51738905,8.1
3,Arrival,2016,47000000,100546139,8.0
4,Manchester by the Sea,2016,9000000,47695371,7.9
...,...,...,...,...,...
95,Whiplash,2014,3300000,13092000,8.5
96,Before Midnight,2013,3000000,8114507,7.9
97,Star Wars: Episode VII - The Force Awakens,2015,245000000,936662225,8.1
98,Harry Potter and the Deathly Hallows: Part I,2010,150000000,296347721,7.7


In [27]:
df_movie.nsmallest(4, "IMDb_rating")

,Title,title_year,budget,Gross,IMDb_rating
22,Hugo,2011,170000000,73820094,7.5
46,Scott Pilgrim vs. the World,2010,60000000,31494270,7.5
52,Lone Survivor,2013,40000000,125069696,7.5
6,Doctor Strange,2016,165000000,232641920,7.6


In [28]:
df_movie.nlargest(4, "IMDb_rating")

,Title,title_year,budget,Gross,IMDb_rating
27,Inception,2010,160000000,292568851,8.8
26,Interstellar,2014,165000000,187991439,8.6
95,Whiplash,2014,3300000,13092000,8.5
8,The Dark Knight Rises,2012,250000000,448130642,8.4


- Usando keep='all' todos os itens duplicados são mantidos

In [29]:
df_movie.nsmallest(4, "IMDb_rating", keep='all')

,Title,title_year,budget,Gross,IMDb_rating
22,Hugo,2011,170000000,73820094,7.5
46,Scott Pilgrim vs. the World,2010,60000000,31494270,7.5
52,Lone Survivor,2013,40000000,125069696,7.5
6,Doctor Strange,2016,165000000,232641920,7.6
21,Dawn of the Planet of the Apes,2014,170000000,208543795,7.6
32,The Hunger Games: Catching Fire,2013,130000000,424645577,7.6
37,Rise of the Planet of the Apes,2011,93000000,176740650,7.6
42,Fury,2014,68000000,85707116,7.6
44,Les MisÃ©rables,2012,61000000,148775460,7.6
49,Moneyball,2011,50000000,75605492,7.6


In [30]:
df_movie.nlargest(4, "IMDb_rating", keep='all')

,Title,title_year,budget,Gross,IMDb_rating
27,Inception,2010,160000000,292568851,8.8
26,Interstellar,2014,165000000,187991439,8.6
95,Whiplash,2014,3300000,13092000,8.5
8,The Dark Knight Rises,2012,250000000,448130642,8.4
35,Django Unchained,2012,100000000,162804648,8.4


- É possível especificar mais de uma coluna como critério de ordenação

In [31]:
df_movie.nsmallest(4, ["IMDb_rating", "Gross"])

,Title,title_year,budget,Gross,IMDb_rating
46,Scott Pilgrim vs. the World,2010,60000000,31494270,7.5
22,Hugo,2011,170000000,73820094,7.5
52,Lone Survivor,2013,40000000,125069696,7.5
99,Tucker and Dale vs Evil,2010,5000000,223838,7.6


In [32]:
df_movie.nlargest(4, ["budget", "Gross"])

,Title,title_year,budget,Gross,IMDb_rating
7,Tangled,2010,260000000,200807262,7.8
8,The Dark Knight Rises,2012,250000000,448130642,8.4
9,Captain America: Civil War,2016,250000000,407197282,7.9
97,Star Wars: Episode VII - The Force Awakens,2015,245000000,936662225,8.1


- O método rank permite ranquear o dataset através dos seus dados numéricos

In [33]:
df_aux = pd.DataFrame(data={'Animal': ['cat', 'penguin', 'dog',
                                   'spider', 'snake'],
                        'Number_legs': [4, 2, 4, 8, np.nan]})
df_aux

,Animal,Number_legs
0,cat,4.0
1,penguin,2.0
2,dog,4.0
3,spider,8.0
4,snake,NaN


- O parâmetro method permite desempatar o ranqueamento pela:
    - média do grupo, usando o argumento average
    - valor mais baixo do grupo, usando o argumento min
    - valor mais alto do grupo, usando o argumento max
    - atribuição da ordem que aparece no array, usando o argumento first
    - valor mais baixo do grupo, mas a próxima posição é aumentado em 1, usando o argumento dense

In [34]:
df_aux['average_rank'] = df_aux['Number_legs'].rank(method='average') 
df_aux['min_rank'] = df_aux['Number_legs'].rank(method='min')
df_aux['max_rank'] = df_aux['Number_legs'].rank(method='max')
df_aux['first_rank'] = df_aux['Number_legs'].rank(method='first')
df_aux['dense_rank'] = df_aux['Number_legs'].rank(method='dense')
df_aux

,Animal,Number_legs,average_rank,min_rank,max_rank,first_rank,dense_rank
0,cat,4.0,2.5,2.0,3.0,2.0,2.0
1,penguin,2.0,1.0,1.0,1.0,1.0,1.0
2,dog,4.0,2.5,2.0,3.0,3.0,2.0
3,spider,8.0,4.0,4.0,4.0,4.0,3.0
4,snake,NaN,NaN,NaN,NaN,NaN,NaN


- O parâmetro na_option define como o ranqueamento lida com os valores NaN pela:
    - manutenção do NaN como NaN, usando o argumento keep
    - atribuição do valor mais baixo, usando o argumento top
    - atribuição do valor mais alto, usando o argumento bottom

In [35]:
df_aux['NA_keep'] = df_aux['Number_legs'].rank(na_option='keep')
df_aux['NA_top'] = df_aux['Number_legs'].rank(na_option='top')
df_aux['NA_bottom'] = df_aux['Number_legs'].rank(na_option='bottom')
df_aux

,Animal,Number_legs,average_rank,min_rank,max_rank,first_rank,dense_rank,NA_keep,NA_top,NA_bottom
0,cat,4.0,2.5,2.0,3.0,2.0,2.0,2.5,3.5,2.5
1,penguin,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0
2,dog,4.0,2.5,2.0,3.0,3.0,2.0,2.5,3.5,2.5
3,spider,8.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,4.0
4,snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0



- O parâmetro pct exibe o ranqueamento em forma percentual

In [36]:
df_aux['pct_rank'] = df_aux['Number_legs'].rank(pct=True)
df_aux

,Animal,Number_legs,average_rank,min_rank,max_rank,first_rank,dense_rank,NA_keep,NA_top,NA_bottom,pct_rank
0,cat,4.0,2.5,2.0,3.0,2.0,2.0,2.5,3.5,2.5,0.625
1,penguin,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.250
2,dog,4.0,2.5,2.0,3.0,3.0,2.0,2.5,3.5,2.5,0.625
3,spider,8.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,4.0,1.000
4,snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN
